In [2]:
import sys
sys.path.insert(0, ".")
import pandas as pd
import numpy as np
import pickle
import networkx as nx
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression
from utils.load_numpy_dataset import load_numpy_data
import itertools


In [3]:
# G = nx.read_adjlist('datasets_pp/persona/CA-HepTh_adj.adjlist')
# nx.write_gexf(G, 'graphs_pp/persona/CA-HepTh.gexf')
# print(G)
# G_o = nx.read_gexf('graphs_pp/original/CA-HepTh.gexf')
# print(G_o)
# G = nx.read_adjlist('datasets_pp/persona/Wiki-Vote_adj.adjlist')
# x = nx.write_edgelist(G,'datasets_pp/persona/Wiki-Vote_edges.txt', data=False, delimiter='\t')
# x

In [11]:
dataset_name = 'CA-AstroPh'
path = f"datasets_pp/original/{dataset_name}"
index, train, train_neg, test, test_neg = load_numpy_data(path)

original_num_nodes for CA-AstroPh is: 18772
largest_cc_num_nodes for CA-AstroPh is: 17903
num_pos_test_edges for CA-AstroPh is: 98516
num_neg_test_edges for CA-AstroPh is: 98516
num_pos_train_edges for CA-AstroPh is: 98515
num_neg_train_edges for CA-AstroPh is: 98515


We don't need index because we are concatenating the train and test sets and then exporting them for the splitter embedding. So whatever index we have in the train files becomes the orignal index for us.

[train, test] -> train on Splitter -> Get embeding and json file

In json file the key is persona; while the value is the original node

In [28]:
index_only = index['index']

In [29]:
ttempk =[]
ttempv =[]
for k,v in index_only.items():
    ttempk.append(int(k))
    ttempv.append(int(v))
print(np.max(ttempk),np.max(ttempv))
print(np.min(ttempk),np.min(ttempv))

17094 3347
0 0


In [5]:
random_state = 42

In [6]:
def graph_edge(data_train,data_train_ng,if_neg=True):
    df = pd.DataFrame(data_train,columns=['source','target'])
    df['weight'] = np.ones(shape=(data_train.shape[0],))
    df2=pd.DataFrame(data_train_ng,columns=['source','target'])
    if if_neg:
        df2['weight'] = np.zeros(shape=(data_train_ng.shape[0],))
    else:
        df2['weight'] = np.ones(shape=(data_train_ng.shape[0],))
    df = df.append(df2)
    # shuffle the DataFrame rows
    df = df.sample(frac = 1,random_state=random_state)
    # reset the index
    df = df.reset_index(drop=True)
    G = nx.from_pandas_edgelist(df, create_using=nx.Graph(),edge_attr=True)
    return G,df

In [7]:
G_train,df_train =graph_edge(train,train_neg)
G_test,df_test =graph_edge(test,test_neg)
G_emed,_ = graph_edge(train,test, if_neg= False)

In [8]:
print(len(G_train.nodes()),np.max(G_train.nodes()),np.min(G_train.nodes()))
print(len(G_test.nodes()),np.max(G_test.nodes()),np.min(G_test.nodes()))
print(len(G_train.edges()))
print(len(G_test.edges()))

17903 17902 0
17903 17902 0
197030
197032


In [19]:
df_emb = pd.read_csv(f"datasets_pp/persona/{dataset_name}_embedding.csv")

In [20]:
df_emb.head()

,id,x_0,x_1,x_2,x_3,x_4,x_5,x_6,x_7,x_8,...,x_22,x_23,x_24,x_25,x_26,x_27,x_28,x_29,x_30,x_31
0,0.0,2.019333,-0.869859,-0.667157,0.933709,-0.306853,0.150578,-0.806043,-0.463944,0.138593,...,-0.624177,-0.768576,-0.859855,-0.505623,-0.365207,0.642208,-0.533835,0.941949,0.340984,-0.563558
1,1.0,27.554132,2.337856,-12.453447,8.607243,7.257886,-5.896724,-12.464989,-0.161737,-7.473846,...,-12.165919,6.969401,21.585445,7.567142,2.289157,-10.464566,-22.983303,10.670724,-2.517677,-3.230413
2,2.0,8.596968,-19.169151,10.174797,26.426212,16.454411,-12.152198,0.205369,-3.094630,-2.367658,...,-7.843886,-15.907837,-22.621639,-45.934158,-13.811146,1.265299,-3.884961,1.855569,-18.796535,1.247050
3,3.0,-6.580640,-14.445245,-16.583439,15.503081,5.806692,-6.975155,-11.313016,-0.317003,8.959885,...,-21.245214,3.051149,-26.246595,-3.365825,4.628250,-5.995635,-1.348278,-2.002706,-6.951435,15.932959
4,4.0,-12.363005,-22.332626,-8.627930,-1.863540,-9.896849,-1.302875,16.308657,17.923262,3.875642,...,10.695974,11.944210,-13.020303,7.578579,6.999091,-19.809200,3.401257,-3.817503,12.304391,4.270333


In [47]:
df_emb

,id,x_0,x_1,x_2,x_3,x_4,x_5,x_6,x_7,x_8,...,x_22,x_23,x_24,x_25,x_26,x_27,x_28,x_29,x_30,x_31
0,0.0,0.179000,-1.209954,-0.097229,0.450680,0.776837,0.318360,0.584043,0.183547,0.352138,...,0.514515,0.166592,-0.219874,-0.274627,0.026739,0.154039,-0.229213,-0.383987,-0.201399,-0.015422
1,1.0,0.594044,-3.817321,1.534369,5.485898,1.179489,0.575863,3.462506,0.172048,1.968799,...,2.565850,2.686307,-1.715784,-0.450326,0.028620,2.449965,-0.797341,-0.824620,-0.886800,-1.526546
2,2.0,3.652542,-14.297075,6.767405,18.072826,5.458565,1.333917,11.683047,1.744974,7.867002,...,10.197902,10.902699,-7.599450,-2.545702,1.267072,9.462002,-4.065391,-2.193506,-4.578306,-6.713081
3,3.0,0.063204,-4.794722,1.121584,6.542805,2.936121,1.252926,2.529882,1.744707,1.384490,...,4.243710,2.453219,-3.440484,0.388752,1.786935,2.045190,-0.128965,-1.873160,-0.163365,-0.978579
4,4.0,1.194163,-3.457294,2.295946,5.332276,0.949026,-0.059782,3.685003,-0.170090,2.466925,...,2.935211,3.399929,-1.613463,-1.081507,-0.291616,1.957961,-1.659939,-0.836030,-1.646934,-2.700895
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17424,17424.0,0.578507,-4.493644,1.606484,5.821136,1.605920,0.611730,3.894386,0.864445,1.767222,...,2.393086,3.626225,-1.844981,-1.098742,0.118272,2.324296,-0.837483,-0.224276,-1.886683,-1.762400
17425,17425.0,1.307554,-4.575622,1.494113,4.214084,1.776021,0.741098,3.351401,0.719006,2.416829,...,2.297828,3.195431,-1.704277,-1.174568,0.104185,2.913064,-0.803396,0.033739,-1.627502,-2.454330
17426,17426.0,1.559779,-4.627253,2.522530,4.975298,2.028236,0.890542,3.958652,1.025267,2.093193,...,3.549354,3.873377,-2.868164,-1.283307,-0.329602,3.395266,-1.713413,-1.152231,-1.891290,-2.651237
17427,17427.0,1.491189,-3.229994,2.398685,4.493958,1.920230,0.831553,3.722345,0.979477,1.692609,...,3.392194,3.654365,-2.658621,-1.219992,-0.276286,3.214955,-1.610796,-1.039214,-1.801213,-2.504235


In [21]:
import json
with open(f"datasets_pp/persona/{dataset_name}_personas.json") as f:
    data = json.loads(f.read())

In [22]:
print(data['0'],data['1'])

15510 15510


In [23]:
og_map={}
map_og = {}
temp_og_map=[]
temp_map_og=[]
# k: personas node (they are unique)
# v: original nodes (they are not unique)
for k,v in data.items():
    # mapped nodes to the original
    map_og[int(k)] = int(v)
    og_map [int(v)] = int(k)
    temp_og_map.append(int(k))
    temp_map_og.append(int(v))

In [24]:
print(len(temp_map_og),len(temp_og_map))
print(len(np.unique(temp_og_map)),len(np.unique(temp_map_og)))

25706 25706
25706 17903


In [92]:
map_og

{0: 3521,
 1: 3521,
 2: 3524,
 3: 3524,
 4: 3524,
 5: 3524,
 6: 3524,
 7: 3524,
 8: 3524,
 9: 5739,
 10: 5739,
 11: 5739,
 12: 5739,
 13: 5739,
 14: 5739,
 15: 5739,
 16: 5739,
 17: 5739,
 18: 8396,
 19: 8396,
 20: 522,
 21: 549,
 22: 1046,
 23: 1046,
 24: 1903,
 25: 2028,
 26: 2028,
 27: 2028,
 28: 2028,
 29: 2028,
 30: 2030,
 31: 2030,
 32: 2030,
 33: 2031,
 34: 2031,
 35: 2031,
 36: 2110,
 37: 2110,
 38: 2110,
 39: 2181,
 40: 2181,
 41: 2181,
 42: 2181,
 43: 2181,
 44: 2181,
 45: 2181,
 46: 2181,
 47: 2181,
 48: 2518,
 49: 2518,
 50: 2518,
 51: 2712,
 52: 2786,
 53: 2786,
 54: 2786,
 55: 2786,
 56: 2786,
 57: 2786,
 58: 2786,
 59: 2786,
 60: 2786,
 61: 2788,
 62: 2788,
 63: 2788,
 64: 2788,
 65: 2788,
 66: 3065,
 67: 3065,
 68: 3303,
 69: 3303,
 70: 3303,
 71: 3303,
 72: 3303,
 73: 3577,
 74: 3577,
 75: 3750,
 76: 3750,
 77: 3782,
 78: 3782,
 79: 3782,
 80: 3782,
 81: 3782,
 82: 3782,
 83: 4064,
 84: 4064,
 85: 4998,
 86: 4998,
 87: 4999,
 88: 4999,
 89: 4999,
 90: 4999,
 91: 4999,


In [25]:

org_nodes = set(map_og.values())

# mapped nodes are the personas nodes and personas nodes are unique
mappedNodes_fromOrg = {}
for n in org_nodes:
    mappedNodes_fromOrg[n] = [k for k in map_og.keys() if map_og[k] == n]

In [26]:
mappedNodes_fromOrg[3521]

[25513, 25514]

In [88]:
og_map[0]

4162

In [82]:
print(np.max(temp_map_og),np.max(temp_og_map))

17428 9876


In [31]:
def aggregate(u,v):
    try:
        res = np.dot(u,v)/(np.linalg.norm(u) * np.linalg.norm(v))
    except:
        res = 0.0
    return res
def edge_embed(u,v):
    # map_og: mapped nodes/personas to the original
    # u,v are the original nodes
    # nodes1 contains all personas nodes for u
    nodes1 = mappedNodes_fromOrg[u]
    # nodes2 contains all personas nodes for v
    nodes2 = mappedNodes_fromOrg[v]
    all_prod = []
    # here u1,v1 are the nodes form personas
    for u1,v1 in itertools.product(nodes1, nodes2):
        # df_emb containts the personas nodes
        e1 = df_emb.iloc[u1,1:]
        e2 = df_emb.iloc[v1,1:]
        all_prod.append(aggregate(e1,e2))
    
    if all_prod:
        res = np.max(all_prod)
    else:
        res = 0.0
    return res
    
def splitter_edgeEmbedding(G):
    X=[]
    y = []
    for u,v in G.edges():
        lbl = G[u][v]["weight"]
        y.append(lbl)
        
        X.append(edge_embed(u,v))
    X = np.asarray(X)
    return X,y

In [32]:
X_train,y_train = splitter_edgeEmbedding(G_train)
X_test,y_test = splitter_edgeEmbedding(G_test)

In [33]:
try:
    X_train.shape[1]
except:
    X_train = np.expand_dims(X_train,axis=1)
    X_test = np.expand_dims(X_test,axis=1)
print(X_train.shape,X_test.shape)

(197030, 1) (197032, 1)


In [34]:
print(np.max(X_train),np.min(X_train))

0.9976069881707404 -0.6287246561545531


In [35]:
clf = LogisticRegression(random_state=random_state).fit(X_train, y_train)

In [36]:
# train score
score_train = roc_auc_score(y_true=y_train, y_score=clf.predict(X_train),  average='micro')
# test score
score_test = roc_auc_score(y_true=y_test, y_score=clf.predict(X_test),  average='micro')

print(f"\n ROC-Score (train): {score_train}\n ROC-Score (test): {score_test}")


 ROC-Score (train): 0.9253869969040247
 ROC-Score (test): 0.9256262942060173


In [46]:
u = mappedNodes_fromOrg[600]
print(u)

[5401, 5402, 5403, 5404]


In [48]:
v=mappedNodes_fromOrg[900]
print(v)

[7549, 7550]


In [49]:
for a,b in itertools.product(u, v):
    print(a,b)

5401 7549
5401 7550
5402 7549
5402 7550
5403 7549
5403 7550
5404 7549
5404 7550


In [63]:
df_test =pd.DataFrame(columns= ['Dataset Name','Embedding Dimension','ROC Train score','ROC Test score'],index=None     )

In [64]:
df_test.to_csv('Results/splitter_results.csv',index = False)

In [65]:
df_check = pd.read_csv('Results/splitter_results.csv')
print(df_check)

Empty DataFrame
Columns: [Dataset Name, Embedding Dimension, ROC Train score, ROC Test score]
Index: []


In [66]:
df_check.head()

,Dataset Name,Embedding Dimension,ROC Train score,ROC Test score
